In [12]:
#Importing the same libraries as earlier
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk import word_tokenize
from nltk import sent_tokenize

from collections import Counter, defaultdict
from datetime import datetime
import re
import math

import warnings
warnings.filterwarnings('ignore')

In [13]:
#Downloading the relevant libraries and dependencies in NLTK module for preprocessing
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [14]:
#Here we will import the processed dataset for demonstration
data = pd.read_csv('/content/drive/MyDrive/Datasets/Indian Financial News Headlines/data/processed/processed_data.csv')

In [15]:
data.head()

,Unnamed: 0,Date,Title,Description,Year,Clean_Title,Clean_Desc,Combined_Text,Combined_Tag_Tokens,Title_Noun_Count,Title_Adverb_Count,Title_Adjective_Count,Title_Verb_Count,Jaccard_Index,num_words,num_unique_words,num_chars,num_words_upper,num_words_title,mean_word_len
0,0,2020-05-26,"ATMs to become virtual bank branches, accept d...","Close to 14.6 per cent (or 35,000) of the 240,...",2020,ATMs become virtual bank branch accept deposit...,close to 14 6 per cent or 35 000 of the 240 00...,ATMs become virtual bank branch accept deposit...,"[('ATMs', 'NNP'), ('become', 'VBP'), ('virtual...",11,2,2,4,0.028571,36,34,187,0,1,4.222222
1,1,2020-05-26,IDFC First Bank seniors to forgo 65% of bonus ...,"V Vaidyanathan, managing director and chief ex...",2020,IDFC First Bank senior forgo 65 bonus amid Cov...,V Vaidyanathan managing director and chief exe...,IDFC First Bank senior forgo 65 bonus amid Cov...,"[('IDFC', 'NNP'), ('First', 'NNP'), ('Bank', '...",13,0,2,4,0.000000,28,28,172,1,4,5.178571
2,2,2020-05-25,"Huge scam in YES Bank for many years, says Enf...",Rana Kapoor's wife also charged with abetting ...,2020,huge scam YES Bank many year say Enforcement D...,Rana Kapoor s wife also charged with abetting ...,huge scam YES Bank many year say Enforcement D...,"[('huge', 'JJ'), ('scam', 'NNS'), ('YES', 'NNP...",9,1,2,4,0.000000,17,17,107,1,3,5.352941
3,3,2020-05-24,"Bank of Maharashtra sanctioned Rs 2,789 cr in ...",The bank said it was now gearing up to extend ...,2020,Bank Maharashtra sanctioned Rs 2 789 cr loan M...,the bank said it wa now gearing up to extend t...,Bank Maharashtra sanctioned Rs 2 789 cr loan M...,"[('Bank', 'NNP'), ('Maharashtra', 'NNP'), ('sa...",13,1,1,7,0.000000,31,30,183,0,7,4.935484
4,4,2020-05-23,DCB Bank's profit before tax declines 37.6% to...,Net profit for the financial year ended March ...,2020,DCB Bank profit tax decline 37 6 Rs 93 84 cror...,net profit for the financial year ended March ...,DCB Bank profit tax decline 37 6 Rs 93 84 cror...,"[('DCB', 'NNP'), ('Bank', 'NNP'), ('profit', '...",16,2,1,2,0.129032,36,30,171,2,5,3.777778


In [16]:
#Again dropping the unnanmed column as before
data.drop(['Unnamed: 0'], axis=1, inplace=True)

In [17]:
data.shape

(11995, 19)

In [18]:
#Firstly, we will spligt the dataset into training and test sets 
data_train = data[ : 10000]
data_test = data[10000 : ]

In [19]:
#I have to maintain a count of the words in vocab for applying the Turney's algorithm for our dataset
#I will be maintaing a Python dictionary where the corpus' tokens will be keys and their corresponding count values

def def_value():
  return "Not Present"

vocab_dict = defaultdict(def_value)

def cnt_vocab(messages):

  for i, message in enumerate(messages):
    messages[i] = nltk.word_tokenize(messages[i])
    text = messages[i]

    for word in text:
      if vocab_dict[word] == "Not Present":
        vocab_dict[word] = 1
      else:
        vocab_dict[word] += 1
  return vocab_dict

In [20]:
#Here we define a dictionary data structure which acts as a frequency Mapper for all tokens in the corpus
messages, token_dict = [], defaultdict(def_value)
for i in range(len(data_train['Combined_Text'])):
  text = data_train['Combined_Text'][i]
  messages.append(text)

token_dict = cnt_vocab(messages)

**Turney's Thumbs-Up or Thumbs-Down algorithm:**


Turney's Thumbs-Up or Thumbs-Down algorithm is stated below:
1. First a POS(Parts of Speech) tagger is applied to the review. For each review, two consecutive words are extracted if their tags conform to any of the patterns in the manner as would be described by the rules as stated in the function.
2. Second step is to estimate the semantic orientation of the extracted phrases, using PMI-IR algorithm. Estimate the semantic orientation of the extracted phrases, using the PMI-IR algorithm. Uses mutual information as a strength of semantic orientation between two words.
3. The third step is to calculate the average semantic orientation of the phrases in the given review and classify the review as recommended if the average is positive and otherwise not recommended.



**1st step implementation - Pattern-based POS tag extraction**

In [21]:
'''
  elif (postag[k][1]=='VB') and (postag[k+1][1]=='NN'):  #new rule added
      tag_pattern.append("".join(postag[k][0]) + " " + "".join(postag[k+1][0]))

  elif (postag[k][1]=='VBN') and (postag[k+1][1]=='NN'):  #new rule added
      tag_pattern.append("".join(postag[k][0]) + " " + "".join(postag[k+1][0]))
'''

def find_pattern(row):
  text = str(row['Combined_Text'])
  postag = nltk.pos_tag(nltk.word_tokenize(text))
  tag_pattern = []

  for k in range(0, len(postag)-2):
    if (postag[k][1]=='JJ' and (postag[k+1][1]=='NN' or postag[k+1][1]=='NNS')):
      tag_pattern.append("".join(postag[k][0]) + " " + "".join(postag[k+1][0]))

    elif (postag[k][1]=='RB' or postag[k][1]=='RBR' or postag[k][1]=='RBS') and (postag[k+1][1]=='JJ') and (postag[k+2][1]!='NN' or postag[k+2][1]!='NNS'):
      tag_pattern.append("".join(postag[k][0]) + " " + "".join(postag[k+1][0]))

    elif (postag[k][1]=='JJ') and (postag[k+1][1]=='JJ') and (postag[k+2][1]!='NN' or postag[k+2][1]!='NNS'):
      tag_pattern.append("".join(postag[k][0]) + " " + "".join(postag[k+1][0]))

    elif (postag[k][1]=='NN') and (postag[k+1][1]=='VBG'): #example - profit declines
      tag_pattern.append("".join(postag[k][0]) + " " + "".join(postag[k+1][0]))

    elif (postag[k][1]=='NN') and (postag[k+1][1]=='VBD'): #example - profit declined
      tag_pattern.append("".join(postag[k][0]) + " " + "".join(postag[k+1][0]))

    elif (postag[k][1]=='VBG') and (postag[k+1][1]=='NN'): #example - company grows
      tag_pattern.append("".join(postag[k][0]) + " " + "".join(postag[k+1][0]))

    elif (postag[k][1]=='VBD') and (postag[k+1][1]=='NN'): #example - company dropped
      tag_pattern.append("".join(postag[k][0]) + " " + "".join(postag[k+1][0]))

    elif (postag[k][1]=='JJS') and (postag[k+1][1]=='NN'): #example - top loser
      tag_pattern.append("".join(postag[k][0]) + " " + "".join(postag[k+1][0]))

    elif (postag[k][1]=='NN' or postag[k][1]=='NNS') and (postag[k+1][1]=='JJ') and (postag[k+2][1]!='NN' or postag[k+2][1]!='NNS'):
       tag_pattern.append("".join(postag[k][0]) + " " + "".join(postag[k+1][0]))

    elif (postag[k][1]=='RB' or postag[k][1]=='RBR' or postag[k][1] == 'RBS') and (postag[k+1][1]=='VB' or postag[k+1][1]=='VBD' or postag[k+1][1]=='VBN' or postag[k+1][1]=='VBG'):
      tag_pattern.append("".join(postag[k][0]) + " " + "".join(postag[k+1][0]))

  return tag_pattern

In [22]:
data_train['Phrases'] = data_train.apply(lambda row : find_pattern(row), axis=1)

In [23]:
#Appending the total text to the end of sentence
total_text = []
for row in range(len(data_train.index)):
  total_text.append(' '.join(str(text) for text in data_train.loc[row, ['Combined_Text']]))

**2nd step - Capture the semantic orientation of extracted phrases, using PMI-IR algorithm**

In [24]:
#How to capture the polarity of a phrase? Positive phrases co-occur more with "excellent", whereas Negative phrases co-occur more with "poor"
#Further description about the algorithm is given in Readme file
'''
NEAR operator has been derived from the following resource:
https://github.com/andelacolak/Turney-algorithm---sentiment-analysis/blob/master/turney_algorithm.py

'''
def near_operator(phrase, word, text_total):

  try:
    #We are using a 
    string= word+r'\W+(?:\w+\W+){0, 500}?'+phrase+r'|'+phrase+r'\W+(?:\w+\W+){0,500}?'+word
    #Try out different ways and different string patterns

    freq_words_in_range = len(re.findall(string, text_total))
    return freq_words_in_range

  except:
    return 0


For each extracted phrase algorithm goes through all the train files and measure their coexistance with the words "great" and "poor". The coexistance is measured by calculating how many times the phrase and word (great or poor) occur at the distance x and less. Higher the x, greater the likelihood that the word does not apply to the phrase.



In [25]:
#The positive keyword "boost" has been chosen for measuring the positive co-existence of extracted phrases with the keyword. The word has been chosen after some relevant sections
#of the corpora.  

def pos_phrase_hits(row, text_total):
  phrases = row['Phrases']
  phrases_length = len(row['Phrases'])
  pos_phrase_hits = [0.01] * phrases_length

 
  for ind, phrase in enumerate(phrases):
    pos_phrase_hits[ind] = near_operator(phrase, "boost", text_total)
    
  return pos_phrase_hits

In [26]:
def neg_phrase_hits(row, text_total):
  phrases = row['Phrases']
  phrases_length = len(row['Phrases'])
  neg_phrase_hits = [0.01] * phrases_length

  for ind, phrase in enumerate(phrases):
    neg_phrase_hits[ind] = near_operator(phrase, "decline", text_total)
    
  return neg_phrase_hits

In [27]:
#Merging the corpora 
text_total = '.'.join(total_text)

In [28]:
data_train['Positive_Hits'] = data_train.apply(lambda row : pos_phrase_hits(row, text_total), axis=1)

In [29]:
data_train['Negative_Hits'] = data_train.apply(lambda row : neg_phrase_hits(row, text_total), axis=1)

In [30]:
data_train.to_csv('/content/drive/MyDrive/Datasets/Indian Financial News Headlines/data/processed/calculate_pmi.csv')

**3rd Step - Calcualting PMI**

In [31]:
t_train = pd.read_csv('/content/drive/MyDrive/Datasets/Indian Financial News Headlines/data/processed/calculate_pmi.csv')
t_train.drop(['Unnamed: 0'], axis=1, inplace=True)

In [32]:
#Calculating the total positive hits for extracted phrases
def total_pos_hits(row, pos_hits, neg_hits):
  phrases = row['Phrases']
  pos_hits += (text_total.count("boost") * len(phrases))
  return pos_hits

In [33]:
#Calculating the total negative hits for extracted phrases
def total_neg_hits(row, pos_hits, neg_hits):
  phrases = row['Phrases']
  neg_hits += (text_total.count("decline") * len(phrases))
  return neg_hits

In [34]:
#Define the threshold for positive as well as negative hits. One can hyperparameterzize and fine-tune it as per one's requirements
pos_hits = 0.01
neg_hits = 0.01

In [35]:
final_pos_hits = t_train.apply(lambda row : total_pos_hits(row, pos_hits, neg_hits), axis=1)
final_neg_hits = t_train.apply(lambda row : total_neg_hits(row, pos_hits, neg_hits), axis=1)

In [36]:
final_pos_hits = np.round(sum(final_pos_hits))
final_neg_hits = np.round(sum(final_neg_hits))

In [37]:
print('The number of positive hits are {} and the number of negative hits are {}'.format(final_pos_hits, final_neg_hits))

The number of positive hits are 57413428.0 and the number of negative hits are 52799857.0


In [38]:
type(final_pos_hits)

numpy.float64

In [39]:
#Just testing it out
regex = '\d+' #The pattern is used to detect digits
re.findall(regex, t_train['Positive_Hits'][0])

['0', '3']

In [40]:
def calculate_sentiment(row, final_pos_hits, final_neg_hits):
  phrases = list(row['Phrases'])
  pos_hit_list, neg_hit_list = [], []
  regex = '\d+' 

  pos_hit_list = re.findall(regex, row['Positive_Hits'])
  neg_hit_list = re.findall(regex, row['Negative_Hits'])

  #Now we will be calcualting the average semantic orientation of each phrase
  #We will predefine the average semantic orientation as 0.0
  
  avg_so = 0.0

  try:
    if (len(pos_hit_list)!=0 and len(neg_hit_list)!=0):
      so = [0] * len(pos_hit_list)

      for i in range(len(pos_hit_list)):

        if (pos_hit_list[i] != 0 and neg_hit_list[i] != 0):
          so[i] = np.log2((float(pos_hit_list[i]) * float(final_neg_hits)) / (float(neg_hit_list[i]) * float(final_pos_hits)))

        else:
          so[i] = 0

      avg_so = sum(so)/len(so)

      if np.round(avg_so) > 0.0:
        avg_so = 1.0

  except Exception as e:
    print(str(e))
      
  return avg_so

In [41]:
t_train['Label'] = t_train.apply(lambda row : calculate_sentiment(row, final_pos_hits, final_neg_hits), axis=1)

Streaming output truncated to the last 5000 lines.
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float 

In [42]:
t_train.head(n=10)

,Date,Title,Description,Year,Clean_Title,Clean_Desc,Combined_Text,Combined_Tag_Tokens,Title_Noun_Count,Title_Adverb_Count,Title_Adjective_Count,Title_Verb_Count,Jaccard_Index,num_words,num_unique_words,num_chars,num_words_upper,num_words_title,mean_word_len,Phrases,Positive_Hits,Negative_Hits,Label
0,2020-05-26,"ATMs to become virtual bank branches, accept d...","Close to 14.6 per cent (or 35,000) of the 240,...",2020,ATMs become virtual bank branch accept deposit...,close to 14 6 per cent or 35 000 of the 240 00...,ATMs become virtual bank branch accept deposit...,"[('ATMs', 'NNP'), ('become', 'VBP'), ('virtual...",11,2,2,4,0.028571,36,34,187,0,1,4.222222,"['virtual bank', 'new age']","[0, 3]","[1, 4]",-inf
1,2020-05-26,IDFC First Bank seniors to forgo 65% of bonus ...,"V Vaidyanathan, managing director and chief ex...",2020,IDFC First Bank senior forgo 65 bonus amid Cov...,V Vaidyanathan managing director and chief exe...,IDFC First Bank senior forgo 65 bonus amid Cov...,"[('IDFC', 'NNP'), ('First', 'NNP'), ('Bank', '...",13,0,2,4,0.000000,28,28,172,1,4,5.178571,"['senior forgo', 'managing director', 'chief e...","[1, 4, 8, 1]","[1, 10, 6, 1]",-0.347577
2,2020-05-25,"Huge scam in YES Bank for many years, says Enf...",Rana Kapoor's wife also charged with abetting ...,2020,huge scam YES Bank many year say Enforcement D...,Rana Kapoor s wife also charged with abetting ...,huge scam YES Bank many year say Enforcement D...,"[('huge', 'JJ'), ('scam', 'NNS'), ('YES', 'NNP...",9,1,2,4,0.000000,17,17,107,1,3,5.352941,"['huge scam', 'many year', 's wife', 'also cha...","[1, 1, 1, 1]","[1, 1, 1, 1]",-0.120854
3,2020-05-24,"Bank of Maharashtra sanctioned Rs 2,789 cr in ...",The bank said it was now gearing up to extend ...,2020,Bank Maharashtra sanctioned Rs 2 789 cr loan M...,the bank said it wa now gearing up to extend t...,Bank Maharashtra sanctioned Rs 2 789 cr loan M...,"[('Bank', 'NNP'), ('Maharashtra', 'NNP'), ('sa...",13,1,1,7,0.000000,31,30,183,0,7,4.935484,"['bank said', 'now gearing', 'stimulus package']","[9, 1, 2]","[7, 1, 3]",-0.194985
4,2020-05-23,DCB Bank's profit before tax declines 37.6% to...,Net profit for the financial year ended March ...,2020,DCB Bank profit tax decline 37 6 Rs 93 84 cror...,net profit for the financial year ended March ...,DCB Bank profit tax decline 37 6 Rs 93 84 cror...,"[('DCB', 'NNP'), ('Bank', 'NNP'), ('profit', '...",16,2,1,2,0.129032,36,30,171,2,5,3.777778,['financial year'],[19],[18],-0.042852
5,2020-05-23,"Bank of Baroda to advance Rs 12,000 crore to M...","Under the scheme, the government will offer 10...",2020,Bank Baroda advance Rs 12 000 crore MSMEs govt...,under the scheme the government will offer 100...,Bank Baroda advance Rs 12 000 crore MSMEs govt...,"[('Bank', 'NNP'), ('Baroda', 'NNP'), ('advance...",13,0,0,1,0.047619,22,21,126,0,3,4.772727,[],[],[],0.000000
6,2020-05-22,"Retail, MSME loans to get cheaper as RBI cuts ...",HDFC Bank cuts base rate by 55 bps; SBI calls ...,2020,retail MSME loan get cheaper RBI cut repo rate...,HDFC Bank cut base rate by 55 bps SBI call ALC...,retail MSME loan get cheaper RBI cut repo rate...,"[('retail', 'JJ'), ('MSME', 'NNP'), ('loan', '...",13,0,4,4,0.086957,25,22,130,4,1,4.240000,"['repo rate', 'base rate']","[12, 6]","[5, 4]",1.000000
7,2020-05-22,"RBI forecasts economic contraction, announces ...",The six-member MPC also kept policy stance unc...,2020,RBI forecast economic contraction announces sh...,the six member MPC also kept policy stance unc...,RBI forecast economic contraction announces sh...,"[('RBI', 'NNP'), ('forecast', 'VBP'), ('econom...",11,1,6,3,0.000000,28,27,173,2,0,5.214286,"['economic contraction', 'announces sharp', 'a...","[1, 1, 1, 1, 1]","[0, 0, 0, 0, 0]",0.000000
8,2020-05-22,"RBI extends loan moratorium till Aug 31, silen...",The central bank has increased group exposure ...,2020,RBI extends loan moratorium till Aug 31 silent...,the central bank ha increased group exposure l...,RBI extends loan moratorium till Aug 31 s

In [43]:
def format_label(row):
  avg_so = row['Label']

  if avg_so < -0.01:
    avg_so = 0
  else:
    avg_so = 1

  return avg_so

In [44]:
t_train.head(80)

,Date,Title,Description,Year,Clean_Title,Clean_Desc,Combined_Text,Combined_Tag_Tokens,Title_Noun_Count,Title_Adverb_Count,Title_Adjective_Count,Title_Verb_Count,Jaccard_Index,num_words,num_unique_words,num_chars,num_words_upper,num_words_title,mean_word_len,Phrases,Positive_Hits,Negative_Hits,Label
0,2020-05-26,"ATMs to become virtual bank branches, accept d...","Close to 14.6 per cent (or 35,000) of the 240,...",2020,ATMs become virtual bank branch accept deposit...,close to 14 6 per cent or 35 000 of the 240 00...,ATMs become virtual bank branch accept deposit...,"[('ATMs', 'NNP'), ('become', 'VBP'), ('virtual...",11,2,2,4,0.028571,36,34,187,0,1,4.222222,"['virtual bank', 'new age']","[0, 3]","[1, 4]",-inf
1,2020-05-26,IDFC First Bank seniors to forgo 65% of bonus ...,"V Vaidyanathan, managing director and chief ex...",2020,IDFC First Bank senior forgo 65 bonus amid Cov...,V Vaidyanathan managing director and chief exe...,IDFC First Bank senior forgo 65 bonus amid Cov...,"[('IDFC', 'NNP'), ('First', 'NNP'), ('Bank', '...",13,0,2,4,0.000000,28,28,172,1,4,5.178571,"['senior forgo', 'managing director', 'chief e...","[1, 4, 8, 1]","[1, 10, 6, 1]",-0.347577
2,2020-05-25,"Huge scam in YES Bank for many years, says Enf...",Rana Kapoor's wife also charged with abetting ...,2020,huge scam YES Bank many year say Enforcement D...,Rana Kapoor s wife also charged with abetting ...,huge scam YES Bank many year say Enforcement D...,"[('huge', 'JJ'), ('scam', 'NNS'), ('YES', 'NNP...",9,1,2,4,0.000000,17,17,107,1,3,5.352941,"['huge scam', 'many year', 's wife', 'also cha...","[1, 1, 1, 1]","[1, 1, 1, 1]",-0.120854
3,2020-05-24,"Bank of Maharashtra sanctioned Rs 2,789 cr in ...",The bank said it was now gearing up to extend ...,2020,Bank Maharashtra sanctioned Rs 2 789 cr loan M...,the bank said it wa now gearing up to extend t...,Bank Maharashtra sanctioned Rs 2 789 cr loan M...,"[('Bank', 'NNP'), ('Maharashtra', 'NNP'), ('sa...",13,1,1,7,0.000000,31,30,183,0,7,4.935484,"['bank said', 'now gearing', 'stimulus package']","[9, 1, 2]","[7, 1, 3]",-0.194985
4,2020-05-23,DCB Bank's profit before tax declines 37.6% to...,Net profit for the financial year ended March ...,2020,DCB Bank profit tax decline 37 6 Rs 93 84 cror...,net profit for the financial year ended March ...,DCB Bank profit tax decline 37 6 Rs 93 84 cror...,"[('DCB', 'NNP'), ('Bank', 'NNP'), ('profit', '...",16,2,1,2,0.129032,36,30,171,2,5,3.777778,['financial year'],[19],[18],-0.042852
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,2020-05-02,"Discretionary loans, cross-selling likely to r...",Larger challenge is to communicate to borrower...,2020,Discretionary loan cross selling likely reduce...,Larger challenge is to communicate to borrower...,Discretionary loan cross selling likely reduce...,"[('Discretionary', 'NNP'), ('loan', 'NN'), ('c...",7,0,2,5,0.050000,21,20,159,0,1,6.619048,['cross selling'],[1],[2],-1.120854
76,2020-05-01,"IDFC First Bank to boost capital by Rs 2,000 c...",Capital infusion to be supported by IDFC limit...,2020,IDFC First Bank boost capital Rs 2 000 cr pref...,Capital infusion to be supported by IDFC limit...,IDFC First Bank boost capital Rs 2 000 cr pref...,"[('IDFC', 'NNP'), ('First', 'NNP'), ('Bank', '...",17,0,2,4,0.037037,28,26,173,4,10,5.214286,"['cr preferential', 'preferential issueCapital...","[0, 0, 0]","[0, 0, 0]",0.000000
77,2020-05-01,Regulatory benefits for banks helping mutual f...,"On April 27, the RBI had introduced the Rs 50,...",2020,Regulatory benefit bank helping mutual fund in...,on April 27 the RBI had introduced the Rs 50 0...,Regulatory benefit bank helping mutual fund in...,"[('Regulatory', 'NNP'), ('benefit', 'NN'), ('b...",14,0,1,4,0.086957,26,23,149,2,3,4.769231,"['bank helping', 'mutual fund']","[0, 3]","[0, 6]",0.000000
78,2020-04-30,Covid-19: Average card spend falls 25% as shop...,"With malls and shops closed, spending avenues ...",2020,Covid 19 Average card spend fall 

In [45]:
t_train['Label_Final'] = t_train.apply(lambda row : format_label(row), axis=1)

In [46]:
t_train['Label_Final'].value_counts()
#Hence we observe that the number of p

1    8030
0    1970
Name: Label_Final, dtype: int64

**Saving the newly created files in the directory**

In [47]:
data_train.to_csv('/content/drive/MyDrive/Datasets/Indian Financial News Headlines/data/processed/turney2.csv')

In [48]:
t_train.to_csv('/content/drive/MyDrive/Datasets/Indian Financial News Headlines/data/processed/turney_v1.csv')

In [49]:
data_test.to_csv('/content/drive/MyDrive/Datasets/Indian Financial News Headlines/data/processed/test_v1.csv')